# Cosmos Genesis Quickstart

This notebook demonstrates basic usage of the Cosmos Genesis Python SDK.

## Prerequisites

1. Install SDK: `pip install cosmos-genesis-client`
2. Get API key from [cosmosgenesis.com](https://cosmosgenesis.com/#beta-signup)
3. Set environment variable: `export COSMOS_API_KEY=your_key`

In [ ]:
# Install SDK (uncomment if needed)
# !pip install cosmos-genesis-client pandas matplotlib

In [ ]:
from cosmos_genesis import CosmosClient, QueryBuilder
import pandas as pd
import matplotlib.pyplot as plt

# Initialize client
client = CosmosClient()  # Reads COSMOS_API_KEY from environment
print("✓ Connected to Cosmos Genesis")

## 1. List Available Galaxies

In [ ]:
# Get list of galaxies
galaxies = client.list_galaxies()

print(f"Available galaxies: {len(galaxies)}\n")
for galaxy in galaxies:
    print(f"  {galaxy['galaxy_id']}: {galaxy['system_count']:,} systems")

## 2. Query Star Data

In [ ]:
# Query massive stars
galaxy_id = "spiral-sm-2arm-001"  # Use your galaxy ID

df = client.query_to_dataframe(
    galaxy_id=galaxy_id,
    query="""
        SELECT
            system_id,
            stellar_mass_msun,
            luminosity_lsun,
            spectral_type,
            age_myr
        FROM star
        WHERE stellar_mass_msun > 10
        LIMIT 1000
    """
)

print(f"Found {len(df)} massive stars")
df.head()

## 3. Analyze Mass Distribution

In [ ]:
# Plot stellar mass histogram
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(df['stellar_mass_msun'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Stellar Mass (M☉)')
plt.ylabel('Count')
plt.title('Stellar Mass Distribution')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(df['stellar_mass_msun'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Stellar Mass (M☉)')
plt.ylabel('Count')
plt.title('Stellar Mass Distribution (Log Scale)')
plt.yscale('log')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Query Builder Example

In [ ]:
# Use query builder for type-safe queries
query = (
    QueryBuilder()
    .select("system_id", "stellar_mass_msun", "spectral_type")
    .from_table("star")
    .where("spectral_type = 'O'")
    .order_by("stellar_mass_msun", "DESC")
    .limit(10)
)

print("Generated query:")
print(query.build())
print()

# Execute query
results = client.query_galaxy(galaxy_id, query.build())
print(f"Top 10 most massive O-type stars:")
for i, star in enumerate(results, 1):
    print(f"  {i}. {star['system_id']}: {star['stellar_mass_msun']} M☉")

## 5. Summary Statistics

In [ ]:
# Compute statistics
print("Summary Statistics:")
print("=" * 60)
print(f"Total stars queried: {len(df):,}")
print(f"Mass range: {df['stellar_mass_msun'].min():.2f} - {df['stellar_mass_msun'].max():.2f} M☉")
print(f"Mean mass: {df['stellar_mass_msun'].mean():.2f} M☉")
print(f"Median mass: {df['stellar_mass_msun'].median():.2f} M☉")
print()

# Group by spectral type
print("Stars by Spectral Type:")
print(df['spectral_type'].value_counts())

## Next Steps

- **[02_imf_analysis.ipynb](02_imf_analysis.ipynb)**: Validate Kroupa Initial Mass Function
- **[03_stellar_evolution.ipynb](03_stellar_evolution.ipynb)**: Track stars over time
- **[04_exoplanet_stats.ipynb](04_exoplanet_stats.ipynb)**: Analyze exoplanet populations

## Documentation

- **Schema Reference**: [cosmos-schemas](https://github.com/cosmosgenesis/cosmos-schemas)
- **API Docs**: [docs.cosmosgenesis.com](https://docs.cosmosgenesis.com)
- **Support**: info@cosmosgenesis.com